<a href="https://colab.research.google.com/github/chandrusuresh/Hands-On-ML/blob/master/Ch4-TrainingModels/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>